In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [11]:
from IPython.display import display, Markdown

In [3]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [4]:
from llama_index.core import Settings, SQLDatabase
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

In [5]:
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Explore DB

In [6]:
engine = create_engine("sqlite:///Chinook.db")
metadata_obj = MetaData()

In [7]:
sql_database = SQLDatabase(engine)

In [8]:
sql_database.get_usable_table_names()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [9]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=['Album','Track', 'Artist'],
    verbose=True
)

In [10]:
response = sql_query_engine.query("What are some albums? Limit to 5.")

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'Album' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> Artist.['ArtistId'].

Table 'Track' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> MediaType.['MediaTypeId'], ['GenreId'] -> Genre.['GenreId'], ['AlbumId'] -> Album.['AlbumId'].

Table 'Artist' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .
> Table desc str: Table 'Album' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> Artist.['ArtistId'].

Table 'Track' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes 

In [13]:
display(Markdown(response.response))

Here are five albums you might find interesting:

1. For Those About To Rock We Salute You
2. Balls to the Wall
3. Restless and Wild
4. Let There Be Rock
5. Big Ones

In [14]:
response.metadata

{'7392fb20-8a74-4182-ae97-4e5ddf1b4bdc': {'sql_query': 'SELECT Album.Title FROM Album LIMIT 5;',
  'result': [('For Those About To Rock We Salute You',),
   ('Balls to the Wall',),
   ('Restless and Wild',),
   ('Let There Be Rock',),
   ('Big Ones',)],
  'col_keys': ['Title']},
 'sql_query': 'SELECT Album.Title FROM Album LIMIT 5;',
 'result': [('For Those About To Rock We Salute You',),
  ('Balls to the Wall',),
  ('Restless and Wild',),
  ('Let There Be Rock',),
  ('Big Ones',)],
 'col_keys': ['Title']}

In [17]:
prompts_dict = sql_query_engine.get_prompts()

In [19]:
for k, prompt_template in prompts_dict.items():
    print(k)
    print(prompt_template)

response_synthesis_prompt
metadata={'prompt_type': <PromptType.SQL_RESPONSE_SYNTHESIS_V2: 'sql_response_synthesis_v2'>} template_vars=['query_str', 'sql_query', 'context_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='Given an input question, synthesize a response from the query results.\nQuery: {query_str}\nSQL: {sql_query}\nSQL Response: {context_str}\nResponse: '
sql_retriever:text_to_sql_prompt
metadata={'prompt_type': <PromptType.TEXT_TO_SQL: 'text_to_sql'>} template_vars=['dialect', 'schema', 'query_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns gi